## Загрузим нужные библиотеки

In [1]:
from sklearn.metrics import f1_score
# нужные библиотеки
import numpy as np
import pandas as pd
import gc
import random
import re
import typing
import numpy as np
import datetime
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
from sklearn.metrics import roc_auc_score
import numpy as np
import optuna
import catboost as cb
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score,recall_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import ast
from collections.abc import Iterable

import pandas as pd
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task


# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
%matplotlib inline

%matplotlib inline

In [2]:
def ragged_flatten(xs):
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from ragged_flatten(x)
        else:
            yield x
            
def getLatLongitudeFromString(s):
    d = ast.literal_eval(s)
    if "geome" in s:
        l = d['geometries'][1]['coordinates']
        
    if "geome" not in s:
        l = d['coordinates']
        
    k = np.array(list(ragged_flatten(l)))
    
    x = np.mean(k[::2])
    y = np.mean(k[1::2])
    return (x,y,len(k)//2)

In [3]:
df = pd.read_csv("train_dataset.csv")
df.head()

,id,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,nd_mean_2021-05-17,nd_mean_2021-05-21,nd_mean_2021-05-26,nd_mean_2021-05-27,nd_mean_2021-05-29,nd_mean_2021-06-06,nd_mean_2021-06-09,nd_mean_2021-06-13,nd_mean_2021-06-16,nd_mean_2021-06-18,nd_mean_2021-06-19,nd_mean_2021-06-20,nd_mean_2021-06-22,nd_mean_2021-06-25,nd_mean_2021-06-27,nd_mean_2021-07-07,nd_mean_2021-07-08,nd_mean_2021-07-09,nd_mean_2021-07-13,nd_mean_2021-07-15,nd_mean_2021-07-17,nd_mean_2021-07-20,nd_mean_2021-07-26,nd_mean_2021-07-27,nd_mean_2021-07-29,nd_mean_2021-07-31,nd_mean_2021-08-07,nd_mean_2021-08-12,.geo,nd_mean_2021-04-15,nd_mean_2021-04-18,nd_mean_2021-04-25,nd_mean_2021-04-27,nd_mean_2021-04-29,nd_mean_2021-04-30,nd_mean_2021-06-07,nd_mean_2021-06-12,nd_mean_2021-06-28,nd_mean_2021-08-01,nd_mean_2021-08-11,nd_mean_2021-05-01,nd_mean_2021-05-03,nd_mean_2021-05-10,nd_mean_2021-05-15,nd_mean_2021-06-02,nd_mean_2021-06-03,nd_mean_2021-07-04,nd_mean_2021-08-10,nd_mean_2021-08-23,nd_mean_2021-04-20,nd_mean_2021-04-23,nd_mean_2021-05-09,nd_mean_2021-05-19,nd_mean_2021-05-20,nd_mean_2021-06-04,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,crop
0,3536,20,0.072846,0.261778,0.062981,0.104442,0.021096,0.052202,0.158723,0.000000,0.484102,0.201013,0.029708,0.397833,0.384805,0.015936,0.007275,0.441994,0.172777,0.000000,0.412072,0.507029,0.490741,0.549791,0.460996,0.100555,0.706968,0.691171,0.711135,0.731325,0.718093,0.000000,0.147169,0.728053,0.112933,0.727516,0.000000,0.000000,0.658016,"{""type"":""Polygon"",""coordinates"":[[[42.49107269...",0.187150,0.262300,0.077577,0.000476,0.000000,0.000000,0.111821,0.038902,0.155038,0.754859,0.000000,0.121799,0.083970,0.213006,0.358304,0.271128,0.173833,0.014060,0.030883,0.034858,0.063134,0.000000,0.102757,0.425571,0.108999,0.118854,0.000000,0.000000,0.026784,0.126832,0.614770,0.008857,0.081498,3
1,3739,45,0.332928,0.287182,0.097138,0.152467,0.075484,0.000000,0.140326,0.163340,0.519456,0.501486,0.047332,0.448796,0.692873,0.628463,0.016747,0.000000,0.059826,0.000000,0.492250,0.708562,0.668011,0.675501,0.694017,0.654727,0.709037,0.696734,0.268523,0.076741,0.638567,0.676917,0.460940,0.059759,0.073804,0.666918,0.089721,0.184231,0.601685,"{""type"":""Polygon"",""coordinates"":[[[50.25545110...",0.245285,0.349133,0.173578,0.196361,0.015048,0.242130,0.708402,0.150383,0.432990,0.706226,0.618475,0.047408,0.000000,0.160796,0.031104,0.000000,0.040128,0.194295,0.062582,0.000000,0.064671,0.024539,0.000000,0.248935,0.134469,0.208268,0.000000,0.000000,0.111148,0.174914,0.179612,0.113071,0.046997,4
2,1294,28,0.013679,0.007055,0.120804,0.037839,0.259125,0.000000,0.037446,0.506516,0.000000,0.561541,0.000000,0.029533,0.245910,0.650065,0.000000,0.516493,0.000000,0.717313,0.649345,0.740904,0.652131,0.739836,0.595076,0.741171,0.648106,0.241615,0.586071,0.551204,0.522871,0.543462,0.000000,0.172532,0.119380,0.321827,0.411168,0.357296,0.000000,"{""type"":""Polygon"",""coordinates"":[[[37.99698998...",0.006391,0.000000,0.041751,0.169020,0.011260,0.118255,0.056252,0.011727,0.172335,0.000000,0.172080,0.230169,0.024845,0.151504,0.002801,0.011655,0.000000,0.135650,0.045370,0.000000,0.000000,0.112471,0.028961,0.127633,0.084467,0.000000,0.210496,0.130467,0.155225,0.090607,0.054127,0.007437,0.219614,2
3,2859,19,0.105976,0.052500,0.001917,0.011531,0.042316,0.000000,0.000000,0.043604,0.510271,0.511552,0.000000,0.524763,0.707093,0.645025,0.753284,0.000000,0.032089,0.064643,0.543500,0.706039,0.617195,0.712928,0.628014,0.563310,0.591204,0.557154,0.188446,0.466411,0.161966,0.425800,0.094829,0.474245,0.242224,0.000000,0.055983,0.060656,0.327025,"{""type"":""Polygon"",""coordinates"":[[[44.72154181...",0.293425,0.318047,0.072589,0.000000,0.145339,0.007611,0.008742,0.099960,0.229617,0.152193,0.379196,0.046483,0.063252,0.099087,0.412416,0.441948,0.7471

In [4]:
df['mean_latitude'] = [getLatLongitudeFromString(str(s['.geo']))[0] for _,s in tqdm(df.iterrows())]
df['mean_longitude'] = [getLatLongitudeFromString(str(s['.geo']))[1] for _,s in tqdm(df.iterrows())]
#df['edge_num'] = [getLatLongitudeFromString(str(s['.geo']))[2] for _,s in tqdm(df.iterrows())]
#df['log_area'] = np.log10(df['area'])

In [5]:
cols = df.columns
ts_cols = [c for c in cols if "nd_mean" in c]

In [6]:
ts_df = df[sorted(set(ts_cols))]

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from autofeat import FeatureSelector, AutoFeatClassifier
import os
import sys
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from autofeat import AutoFeatClassifier

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [8]:
df_1 = df.copy()
z = df_1[ts_cols].copy()
z[z<-5.00001] = np.NaN
k = z[sorted(ts_cols)].copy()
k = k.interpolate(axis=1)
num_zeros = np.sum(1.0*(z.values<0.0001),axis = -1)
num_small = np.sum(1.0*(z.values<0.1),axis = -1)

df_1[sorted(ts_cols)] = k
log_df = pd.DataFrame(-1*np.log(k+0.0001))
log_df = log_df.set_axis([(s+'_log') for s in k.columns], axis=1, inplace=False)
#df_1 = pd.concat([df_1,log_df],axis = 1)
df_1['num_zeros'] = num_zeros
df_1['num_small'] = num_small

In [9]:
z_long = pd.DataFrame({0: z.fillna(-1).values.flatten(),
                          1: np.arange(z.shape[0]).repeat(z.shape[1])})
print(z_long.shape)
z_long.head()
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import settings
settings_comprehensive = settings.MinimalFCParameters()

X = extract_features(z_long.fillna(-1), column_id=1, impute_function=impute, default_fc_parameters=settings_comprehensive)
print(X.shape)
y = df['crop']

(338100, 2)


Feature Extraction: 100%|██████████| 30/30 [00:00<00:00, 39.14it/s]


(4830, 10)


In [10]:
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import settings
settings_comprehensive = settings.MinimalFCParameters()

X = extract_features(z_long.fillna(-1), column_id=1, impute_function=impute, default_fc_parameters=settings_comprehensive)
print(X.shape)
y = df['crop']
relevant_features = set()


Feature Extraction: 100%|██████████| 30/30 [00:00<00:00, 40.70it/s]


(4830, 10)


In [11]:
for label in y.dropna().unique():
    # select_features работает с бинарной классификацией, поэтому переводим задачу
    # в бинарную для каждого класса и повторяем по всем классам
    y_binary = y.dropna() == label
    X_filtered = select_features(X, y_binary)
    relevant_features = relevant_features.union(set(X_filtered.columns))


In [12]:
df_1 = df.copy()
df_1 = pd.concat([df_1,log_df,X[relevant_features]],axis = 1)
df_1['num_zeros'] = num_zeros
df_1['num_small'] = num_small
for i in [0.2,0.3,0.4,0.5,0.6,0.75]:
    df_1['num_less_than_' + str(i)] = np.sum(1.0*(z.values<i),axis = -1)
df_1.shape

(4830, 162)

In [13]:
import joblib
train_weather = joblib.load('train_weather.joblib')  

In [14]:
from tqdm.auto import trange

In [15]:
weather_df = np.ones((df_1.shape[0],train_weather[0].values.flatten().size))*np.nan
for t in trange(df_1.shape[0]):
    if train_weather[t].size == 35:
        weather_df[t,:] = train_weather[t].values.flatten()
weather_df = pd.DataFrame(weather_df,columns = ['weather_' + str(i) for i in range(35)])
df_1 = pd.concat([df_1,weather_df],axis = 1)


In [16]:
print(df_1.shape)
df_1.head()

(4830, 197)


,id,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,nd_mean_2021-05-17,nd_mean_2021-05-21,nd_mean_2021-05-26,nd_mean_2021-05-27,nd_mean_2021-05-29,nd_mean_2021-06-06,nd_mean_2021-06-09,nd_mean_2021-06-13,nd_mean_2021-06-16,nd_mean_2021-06-18,nd_mean_2021-06-19,nd_mean_2021-06-20,nd_mean_2021-06-22,nd_mean_2021-06-25,nd_mean_2021-06-27,nd_mean_2021-07-07,nd_mean_2021-07-08,nd_mean_2021-07-09,nd_mean_2021-07-13,nd_mean_2021-07-15,nd_mean_2021-07-17,nd_mean_2021-07-20,nd_mean_2021-07-26,nd_mean_2021-07-27,nd_mean_2021-07-29,nd_mean_2021-07-31,nd_mean_2021-08-07,nd_mean_2021-08-12,.geo,nd_mean_2021-04-15,nd_mean_2021-04-18,nd_mean_2021-04-25,nd_mean_2021-04-27,nd_mean_2021-04-29,nd_mean_2021-04-30,nd_mean_2021-06-07,nd_mean_2021-06-12,nd_mean_2021-06-28,nd_mean_2021-08-01,...,0__sum_values,0__absolute_maximum,0__median,0__standard_deviation,0__mean,0__root_mean_square,0__variance,num_zeros,num_small,num_less_than_0.2,num_less_than_0.3,num_less_than_0.4,num_less_than_0.5,num_less_than_0.6,num_less_than_0.75,weather_0,weather_1,weather_2,weather_3,weather_4,weather_5,weather_6,weather_7,weather_8,weather_9,weather_10,weather_11,weather_12,weather_13,weather_14,weather_15,weather_16,weather_17,weather_18,weather_19,weather_20,weather_21,weather_22,weather_23,weather_24,weather_25,weather_26,weather_27,weather_28,weather_29,weather_30,weather_31,weather_32,weather_33,weather_34
0,3536,20,0.072846,0.261778,0.062981,0.104442,0.021096,0.052202,0.158723,0.000000,0.484102,0.201013,0.029708,0.397833,0.384805,0.015936,0.007275,0.441994,0.172777,0.000000,0.412072,0.507029,0.490741,0.549791,0.460996,0.100555,0.706968,0.691171,0.711135,0.731325,0.718093,0.000000,0.147169,0.728053,0.112933,0.727516,0.000000,0.000000,0.658016,"{""type"":""Polygon"",""coordinates"":[[[42.49107269...",0.187150,0.262300,0.077577,0.000476,0.000000,0.000000,0.111821,0.038902,0.155038,0.754859,...,15.891094,0.754859,0.115894,0.245319,0.227016,0.334242,0.060181,11.0,29.0,44.0,49.0,52.0,58.0,60.0,69.0,9.7,3.7,15.2,67.8,13.3,1016.3,NaN,17.6,10.9,23.8,50.8,13.0,1013.2,NaN,22.4,15.9,28.8,49.5,11.3,1013.4,NaN,24.8,17.8,31.6,51.0,9.8,1012.1,NaN,24.2,18.2,30.8,61.4,11.1,1014.1,NaN
1,3739,45,0.332928,0.287182,0.097138,0.152467,0.075484,0.000000,0.140326,0.163340,0.519456,0.501486,0.047332,0.448796,0.692873,0.628463,0.016747,0.000000,0.059826,0.000000,0.492250,0.708562,0.668011,0.675501,0.694017,0.654727,0.709037,0.696734,0.268523,0.076741,0.638567,0.676917,0.460940,0.059759,0.073804,0.666918,0.089721,0.184231,0.601685,"{""type"":""Polygon"",""coordinates"":[[[50.25545110...",0.245285,0.349133,0.173578,0.196361,0.015048,0.242130,0.708402,0.150383,0.432990,0.706226,...,18.941434,0.709037,0.174246,0.253821,0.270592,0.371006,0.064425,9.0,26.0,40.0,46.0,48.0,52.0,54.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1294,28,0.013679,0.007055,0.120804,0.037839,0.259125,0.000000,0.037446,0.506516,0.000000,0.561541,0.000000,0.029533,0.245910,0.650065,0.000000,0.516493,0.000000,0.717313,0.649345,0.740904,0.652131,0.739836,0.595076,0.741171,0.648106,0.241615,0.586071,0.551204,0.522871,0.543462,0.000000,0.172532,0.119380,0.321827,0.411168,0.357296,0.000000,"{""type"":""Polygon"",""coordinates"":[[[37.99698998...",0.006391,0.000000,0.041751,0.169020,0.011260,0.118255,0.056252,0.011727,0.172335,0.000000,...,14.879883,0.741171,0.120092,0.242910,0.212570,0.322786,0.059005,13.0,32.0,45.0,51.0,53.0,54.0,62.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2859,19,0.105976,0.052500,0.001917,0.011531,0.042316,0.000000,0.000000,0.043604,0.510271,0.511552,0.000000,0.524763,0.707093,0.645025,0.753284,0.000000,0.032089,0.064643,0.543500,0.706039,0.617195,0.712928,0.628014,0.563310,

In [19]:
N_THREADS = 11 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 137 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 1200 # Time in seconds for automl run
TARGET_NAME = 'crop'

In [20]:
task = Task('multiclass',metric = lambda y_true, y_pred: recall_score(y_true, np.argmax(y_pred,axis =-1),average = 'macro'))

In [21]:
roles = {
    'target': TARGET_NAME,
    'drop': ['id','.geo'],
}

In [22]:
automl = TabularAutoML(task = task, 
                               timeout = TIMEOUT,
                               cpu_limit = N_THREADS,
                               reader_params = {'n_jobs': N_THREADS,'cv':N_FOLDS,'random_state':RANDOM_STATE},
                      general_params = {'nested_cv': False, 'use_algos': [['linear_l2','lgb', 'lgb_tuned']]},)


In [23]:
automl.fit_predict(df_1, roles = roles,verbose = 3)

[14:02:41] Stdout logging level is INFO3.
[14:02:41] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[14:02:41] Task: multiclass

[14:02:41] Start automl preset with listed constraints:
[14:02:41] - time: 1200.00 seconds
[14:02:41] - CPU: 11 cores
[14:02:41] - memory: 16 GB

[14:02:41] Train data shape: (4830, 197)

[14:02:44] Feats was rejected during automatic roles guess: []
[14:02:44] Layer 1 train process start. Time left 1196.68 secs
[14:02:45] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[14:02:45] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[14:02:57] Linear model: C = 1e-05 score = 0.7655951404064811
[14:03:10] Linear model: C = 5e-05 score = 0.8643389208621818
[14:03:29] Linear model: C = 0.0001 score = 0.8823759458253468
[14:03:47] Linear model: C = 0.0005 score = 0.9118828915385028
[14:04:00] Linear model: C = 0.001 score = 0.916848812416746
[14:04:17] Linear model: C = 0.005 score = 0.94352026157

[14:08:20] [700]	valid's multi_logloss: 0.0763137	valid's Opt metric: 0.974315
[14:08:22] [800]	valid's multi_logloss: 0.0711301	valid's Opt metric: 0.976292
[14:08:24] [900]	valid's multi_logloss: 0.0680971	valid's Opt metric: 0.976292
[14:08:26] Trial 1 with hyperparameters {'feature_fraction': 0.6872700594236812, 'num_leaves': 244, 'bagging_fraction': 0.8659969709057025, 'min_sum_hessian_in_leaf': 0.24810409748678125, 'reg_alpha': 2.5361081166471375e-07, 'reg_lambda': 2.5348407664333426e-07} scored 0.9762921926616638 in 0:01:04.232030
[14:08:26] [1]	valid's multi_logloss: 1.9224	valid's Opt metric: 0.310281
[14:08:26] Training until validation scores don't improve for 200 rounds
[14:08:28] [100]	valid's multi_logloss: 0.81822	valid's Opt metric: 0.961922
[14:08:29] [200]	valid's multi_logloss: 0.433504	valid's Opt metric: 0.965121
[14:08:31] [300]	valid's multi_logloss: 0.263772	valid's Opt metric: 0.970225
[14:08:33] [400]	valid's multi_logloss: 0.18357	valid's Opt metric: 0.969151

[14:11:16] Training until validation scores don't improve for 200 rounds
[14:11:23] [100]	valid's multi_logloss: 0.767791	valid's Opt metric: 0.961058
[14:11:31] [200]	valid's multi_logloss: 0.386868	valid's Opt metric: 0.963071
[14:11:46] [300]	valid's multi_logloss: 0.224924	valid's Opt metric: 0.970241
[14:11:53] [400]	valid's multi_logloss: 0.150511	valid's Opt metric: 0.970159
[14:11:57] [500]	valid's multi_logloss: 0.114405	valid's Opt metric: 0.972249
[14:12:01] [600]	valid's multi_logloss: 0.0951241	valid's Opt metric: 0.971175
[14:12:05] Trial 10 with hyperparameters {'feature_fraction': 0.7989499894055425, 'num_leaves': 237, 'bagging_fraction': 0.5442462510259598, 'min_sum_hessian_in_leaf': 0.006080390190296602, 'reg_alpha': 2.5529693461039728e-08, 'reg_lambda': 8.471746987003668e-06} scored 0.9722487301465913 in 0:00:49.739656
[14:12:06] [1]	valid's multi_logloss: 1.91896	valid's Opt metric: 0.415854
[14:12:06] Training until validation scores don't improve for 200 rounds
[1

array([[3.24554043e-04, 1.33906535e-04, 1.14753944e-04, ...,
        1.94427557e-04, 1.89147075e-04, 3.76742915e-04],
       [1.21390694e-04, 4.70715859e-05, 4.71727508e-05, ...,
        9.99623239e-01, 4.53750399e-05, 7.13903210e-05],
       [6.54391624e-05, 6.06599897e-05, 9.99576926e-01, ...,
        1.01236757e-04, 5.77232313e-05, 8.51269433e-05],
       ...,
       [5.76954102e-03, 2.65063741e-03, 2.84768711e-03, ...,
        9.11409035e-03, 2.59311870e-03, 9.74511087e-01],
       [2.09484645e-03, 1.96400564e-03, 1.98881910e-03, ...,
        2.01188051e-03, 9.88180935e-01, 1.86807977e-03],
       [2.37187460e-01, 3.54249123e-03, 3.96736804e-03, ...,
        9.19620618e-02, 3.42466659e-03, 6.56565785e-01]], dtype=float32)

In [24]:
df_test = pd.read_csv("test_dataset.csv")
df_test['mean_latitude'] = [getLatLongitudeFromString(str(s['.geo']))[0] for _,s in df_test.iterrows()]
df_test['mean_longitude'] = [getLatLongitudeFromString(str(s['.geo']))[1] for _,s in df_test.iterrows()]
##df_test['edge_num'] = [getLatLongitudeFromString(str(s['.geo']))[2] for _,s in df_test.iterrows()]
#df_test['log_area'] = np.log10(df_test['area'])
df_ans = df_test.copy()


In [25]:
z = df_ans[ts_cols].copy()
z[z<-5.00001] = np.NaN
k = z[sorted(ts_cols)].copy()
k = k.interpolate(axis=1)
num_zeros = np.sum(1.0*(z.values<0.0001),axis = -1)
df_ans[sorted(ts_cols)] = k

log_df = pd.DataFrame(-1*np.log(k+0.0001))
log_df = log_df.set_axis([(s+'_log') for s in k.columns], axis=1, inplace=False)
df_ans = pd.concat([df_ans,log_df],axis = 1)
df_ans['num_zeros'] = num_zeros

In [26]:
test_weather = joblib.load('test_weather.joblib')  

In [27]:
weather_df = np.ones((df_ans.shape[0],test_weather[0].values.flatten().size))*np.nan
for t in trange(df_ans.shape[0]):
    if test_weather[t].size == 35:
        weather_df[t,:] = test_weather[t].values.flatten()
weather_df = pd.DataFrame(weather_df,columns = ['weather_' + str(i) for i in range(35)])
df_ans = pd.concat([df_ans,weather_df],axis = 1)


In [28]:
z_long = pd.DataFrame({0: z.fillna(-1).values.flatten(),
                          1: np.arange(z.shape[0]).repeat(z.shape[1])})
print(z_long.shape)
z_long.head()
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import settings

X_test = extract_features(z_long.fillna(-1), column_id=1, impute_function=impute, default_fc_parameters=settings_comprehensive)
print(X_test.shape)

(144970, 2)


Feature Extraction: 100%|██████████| 30/30 [00:00<00:00, 90.39it/s]


(2071, 10)


In [29]:
df_test = pd.concat([df_ans,X_test[relevant_features]],axis = 1)
df_test['num_small'] = np.sum(1.0*(z.values<0.1),axis = -1)
for i in [0.2,0.3,0.4,0.5,0.6,0.75]:
    df_test['num_less_than_' + str(i)] = np.sum(1.0*(z.values<i),axis = -1)
#df_test = df_ans.copy()

In [30]:
probs = np.array(automl.predict(df_test).data)
preds = np.argmax(probs,axis = -1)
labels = preds.copy()

In [31]:
confidence = np.max(probs,axis = -1)

In [32]:
pd.DataFrame(probs).to_csv('probs_0.971')

In [33]:
df_submission = pd.read_csv("./sample_solution.csv")

In [34]:
df_submission['crop'] = labels

In [35]:
df_submission.to_csv('lama_ndvi_prefinal3.csv',index = False)

In [41]:
###final blend
p_1 = pd.read_csv('probs_0.971').values[:,1:]
p_2 = pd.read_csv('probs_0.972').values[:,1:]
p_3 = pd.read_csv('probs_0.974').values[:,1:]

In [43]:
total_probs = (p_1*0.3 + p_2*0.3 +p_3*0.4)
preds = np.argmax(total_probs,axis = -1)
labels = preds.copy()
df_submission['crop'] = labels
df_submission.to_csv('lama_ndvi_final.csv',index = False)